# Proyecto de Investigación Estructura de datos
### INTEGRANTES: 
-	Damaris Betsabe Cando Arauz 
-	Diego Fernando Cuenca Salinas 
-	Jose Daniel Guaman Bermeo 
-	Gabriel Alejandro Niama Martínez
### PARALELO: SEGUNDO "B"

## 🛒 Simulador de Compras en Línea


Este proyecto implementa un __simulador de compras en línea__, utilizando varias estructuras de datos y algoritmos vistos en la materia de Estructura de Datos. El usuario puede seleccionar productos, agregarlos a un carrito, deshacer acciones, realizar compras, ver historial de fechas y calcular rutas de entrega entre tiendas.

### 📌 Estructuras de Datos Implementadas
__1. Lista Enlazada (Carrito de Compra)__
+ Se usó una lista enlazada simple para representar el carrito de compras.

+ Cada nodo almacena un producto y un puntero al siguiente.

+ Permite agregar y eliminar productos de forma eficiente.

__2. Pila (Historial de Acciones)__
+ Se implementó una pila para guardar las acciones realizadas (agregar o eliminar producto).

+ Permite deshacer la última acción gracias a la propiedad LIFO (Last In, First Out).

__3. Cola (Cola de Atención)__
+ Al realizar una compra, el carrito se encola en una cola FIFO.

+ Se atienden las compras en el mismo orden en que fueron realizadas.

__4. Grafo (Red de Tiendas con Dijkstra)__
+ Se modeló una red de tiendas usando un grafo no dirigido.

+ Se aplicó el algoritmo de Dijkstra para calcular las distancias más cortas entre tiendas, simulando rutas de entrega.

__5. Historial con Fecha (Lista Cronológica)__
+ Se guarda la fecha y hora exacta de cada compra realizada.

+ Esto permite tener un historial ordenado cronológicamente.



## 1. CÓDIGO COMPLETO DEL SIMULADOR

In [8]:
from datetime import datetime
import heapq

# --- Producto ---
class Producto:
    def __init__(self, id, nombre, precio):
        self.id = id
        self.nombre = nombre
        self.precio = precio

    def __str__(self):
        return f"ID: {self.id} | {self.nombre} | ${self.precio}"

# --- Nodo del carrito (lista enlazada) ---
class NodoCarrito:
    def __init__(self, producto):
        self.producto = producto
        self.siguiente = None

# --- Carrito (Lista enlazada) ---
class CarritoDeCompras:
    def __init__(self):
        self.cabeza = None

    def agregar_producto(self, producto):
        nodo = NodoCarrito(producto)
        nodo.siguiente = self.cabeza
        self.cabeza = nodo

    def eliminar_producto(self, id_producto):
        actual = self.cabeza
        anterior = None
        while actual:
            if actual.producto.id == id_producto:
                if anterior:
                    anterior.siguiente = actual.siguiente
                else:
                    self.cabeza = actual.siguiente
                return actual.producto
            anterior = actual
            actual = actual.siguiente
        return None

    def mostrar_carrito(self):
        actual = self.cabeza
        if not actual:
            print("El carrito está vacío.")
        else:
            while actual:
                print(actual.producto)
                actual = actual.siguiente

# --- Acciones (Pila para deshacer) ---
class Accion:
    def __init__(self, tipo, producto):
        self.tipo = tipo
        self.producto = producto

class HistorialAcciones:
    def __init__(self):
        self.pila_acciones = []

    def push(self, accion):
        self.pila_acciones.append(accion)

    def pop(self):
        if self.pila_acciones:
            return self.pila_acciones.pop()
        return None

    def deshacer_ultima_accion(self, carrito):
        accion = self.pop()
        if not accion:
            print("No hay acciones para deshacer.")
            return
        if accion.tipo == "agregar":
            carrito.eliminar_producto(accion.producto.id)
            print(f"Deshecho: agregado {accion.producto.nombre}")
        elif accion.tipo == "eliminar":
            carrito.agregar_producto(accion.producto)
            print(f"Deshecho: eliminado {accion.producto.nombre}")

# --- Cola de atención ---
class ColaAtencion:
    def __init__(self):
        self.elementos = []

    def encolar(self, carrito):
        self.elementos.append(carrito)
        print("Compra encolada para atención.")

    def desencolar(self):
        if self.elementos:
            carrito = self.elementos.pop(0)
            print("Compra atendida:")
            carrito.mostrar_carrito()
        else:
            print("No hay compras en espera.")

    def mostrar_cola(self):
        print(f"Hay {len(self.elementos)} compra(s) en espera.")

# --- MergeSort para productos ---
def merge_sort_productos(lista, clave="precio"):
    if len(lista) <= 1:
        return lista
    medio = len(lista) // 2
    izquierda = merge_sort_productos(lista[:medio], clave)
    derecha = merge_sort_productos(lista[medio:], clave)
    return merge(izquierda, derecha, clave)

def merge(izq, der, clave):
    resultado = []
    i = j = 0
    while i < len(izq) and j < len(der):
        if getattr(izq[i], clave) < getattr(der[j], clave):
            resultado.append(izq[i])
            i += 1
        else:
            resultado.append(der[j])
            j += 1
    resultado.extend(izq[i:])
    resultado.extend(der[j:])
    return resultado

# --- Grafo con Dijkstra ---
class GrafoTiendas:
    def __init__(self):
        self.vertices = {}

    def agregar_tienda(self, nombre):
        if nombre not in self.vertices:
            self.vertices[nombre] = {}

    def agregar_ruta(self, origen, destino, distancia):
        self.vertices[origen][destino] = distancia
        self.vertices[destino][origen] = distancia

    def dijkstra(self, inicio):
        distancias = {v: float('inf') for v in self.vertices}
        distancias[inicio] = 0
        cola = [(0, inicio)]

        while cola:
            distancia_actual, actual = heapq.heappop(cola)

            for vecino, peso in self.vertices[actual].items():
                nueva_distancia = distancia_actual + peso
                if nueva_distancia < distancias[vecino]:
                    distancias[vecino] = nueva_distancia
                    heapq.heappush(cola, (nueva_distancia, vecino))
        return distancias

# --- SIMULADOR COMPLETO ---
class SimuladorCompras:
    def __init__(self):
        self.catalogo_productos = [
            Producto(1, "Laptop", 1200),
            Producto(2, "Mouse", 25),
            Producto(3, "Teclado", 45),
            Producto(4, "Monitor", 300)
        ]
        self.carrito = CarritoDeCompras()
        self.historial = HistorialAcciones()
        self.cola_atencion = ColaAtencion()
        self.historial_fechas = []

        self.grafo = GrafoTiendas()
        self.grafo.agregar_tienda("Tienda A")
        self.grafo.agregar_tienda("Tienda B")
        self.grafo.agregar_tienda("Tienda C")
        self.grafo.agregar_ruta("Tienda A", "Tienda B", 5)
        self.grafo.agregar_ruta("Tienda A", "Tienda C", 10)
        self.grafo.agregar_ruta("Tienda B", "Tienda C", 3)

    def mostrar_menu(self):
        while True:
            print("\n--- Menú ---")
            print("1. Ver productos disponibles")
            print("2. Agregar producto al carrito")
            print("3. Eliminar producto del carrito")
            print("4. Ver carrito")
            print("5. Deshacer última acción")
            print("6. Realizar compra")
            print("7. Ver cola de atención")
            print("8. Atender compra")
            print("9. Salir")
            print("10. Ver historial de compras")
            print("11. Ver rutas de entrega (Dijkstra)")

            opcion = input("Selecciona una opción: ")

            if opcion == "1":
                self.ver_productos()
            elif opcion == "2":
                self.agregar_producto()
            elif opcion == "3":
                self.eliminar_producto()
            elif opcion == "4":
                self.carrito.mostrar_carrito()
            elif opcion == "5":
                self.historial.deshacer_ultima_accion(self.carrito)
            elif opcion == "6":
                self.realizar_compra()
            elif opcion == "7":
                self.cola_atencion.mostrar_cola()
            elif opcion == "8":
                self.cola_atencion.desencolar()
            elif opcion == "9":
                print("Saliendo del simulador...")
                break
            elif opcion == "10":
                self.ver_historial_fechas()
            elif opcion == "11":
                self.ver_rutas()
            else:
                print("Opción no válida.")

    def ver_productos(self):
        criterio = input("¿Ordenar por 'precio' o 'nombre'? ")
        ordenados = merge_sort_productos(self.catalogo_productos, clave=criterio)
        for p in ordenados:
            print(p)

    def agregar_producto(self):
        try:
            id_prod = int(input("Ingrese el ID del producto a agregar: "))
            producto = next((p for p in self.catalogo_productos if p.id == id_prod), None)
            if producto:
                self.carrito.agregar_producto(producto)
                self.historial.push(Accion("agregar", producto))
                print(f"{producto.nombre} agregado al carrito.")
            else:
                print("Producto no encontrado.")
        except ValueError:
            print("Ingrese un número válido.")

    def eliminar_producto(self):
        try:
            id_prod = int(input("Ingrese el ID del producto a eliminar: "))
            producto = self.carrito.eliminar_producto(id_prod)
            if producto:
                self.historial.push(Accion("eliminar", producto))
                print(f"{producto.nombre} eliminado del carrito.")
            else:
                print("Producto no encontrado en el carrito.")
        except ValueError:
            print("Ingrese un número válido.")

    def realizar_compra(self):
        if not self.carrito.cabeza:
            print("El carrito está vacío. No se puede realizar la compra.")
            return
        self.cola_atencion.encolar(self.carrito)
        fecha = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        self.historial_fechas.append(fecha)
        print(f"Compra registrada en: {fecha}")
        self.carrito = CarritoDeCompras()
        self.historial = HistorialAcciones()

    def ver_historial_fechas(self):
        print("\nHistorial de fechas de compra:")
        for i, fecha in enumerate(self.historial_fechas, 1):
            print(f"{i}. {fecha}")

    def ver_rutas(self):
        inicio = input("Desde qué tienda deseas calcular (Tienda A, B o C): ")
        if inicio in self.grafo.vertices:
            distancias = self.grafo.dijkstra(inicio)
            print(f"Distancias desde {inicio}:")
            for tienda, distancia in distancias.items():
                print(f" - hasta {tienda}: {distancia} km")
        else:
            print("Tienda no válida.")


### ✅ __Funcionalidades del Simulador__
| Funcionalidad              | Estructura Usada              |
| -------------------------- | ----------------------------- |
| Ver catálogo ordenado      | MergeSort                     |
| Agregar/eliminar productos | Lista Enlazada                |
| Deshacer última acción     | Pila                          |
| Realizar compras           | Cola                          |
| Guardar fecha de compra    | Lista (Historial Cronológico) |
| Ver rutas de entrega       | Grafo + Dijkstra              |


## 🚀 Ejecución del simulador

A continuación se crea una instancia del simulador y se ejecuta el menú principal.

In [9]:
# Ejecutar el simulador
simulador = SimuladorCompras()
simulador.mostrar_menu()



--- Menú ---
1. Ver productos disponibles
2. Agregar producto al carrito
3. Eliminar producto del carrito
4. Ver carrito
5. Deshacer última acción
6. Realizar compra
7. Ver cola de atención
8. Atender compra
9. Salir
10. Ver historial de compras
11. Ver rutas de entrega (Dijkstra)
ID: 2 | Mouse | $25
ID: 3 | Teclado | $45
ID: 4 | Monitor | $300
ID: 1 | Laptop | $1200

--- Menú ---
1. Ver productos disponibles
2. Agregar producto al carrito
3. Eliminar producto del carrito
4. Ver carrito
5. Deshacer última acción
6. Realizar compra
7. Ver cola de atención
8. Atender compra
9. Salir
10. Ver historial de compras
11. Ver rutas de entrega (Dijkstra)
Laptop agregado al carrito.

--- Menú ---
1. Ver productos disponibles
2. Agregar producto al carrito
3. Eliminar producto del carrito
4. Ver carrito
5. Deshacer última acción
6. Realizar compra
7. Ver cola de atención
8. Atender compra
9. Salir
10. Ver historial de compras
11. Ver rutas de entrega (Dijkstra)
Laptop agregado al carrito.

--- Me